# Basic working of Google LLM in LangChain

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install langchain-google-genai
!pip install InstructorEmbedding
!pip install torchvision 
!pip install sentence-transformers==2.2.2
!pip install faiss-cpu
!pip install urllib3==1.26.6

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
        model='gemini-1.5-pro',
        google_api_key='your api key here', # get this free api key from https://makersuite.google.com/
        temperature=0.5
)

poem = llm.invoke("Write a 4 line poem of my love for samosa")
print(poem.content)


Golden brown, a crispy shell,
Spicy potatoes, a taste I tell.
With chutney's tang, a perfect bite,
Oh, samosa, my heart's delight! 




# Now let's load data from Codebasics FAQ csv file

In [5]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(
    file_path='codebasics_faqs.csv', 
    encoding="ISO-8859-1", 
    source_column='prompt'
)

docs = loader.load()

# Hugging Face Embeddings

In [ ]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS


instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large",
    query_instruction="Represent the query for retrieval: "
)

vectordb = FAISS.from_documents(
    documents=docs, 
    embedding=instructor_embeddings
)


# Vector store using FAISS

In [15]:
retriever = vectordb.as_retriever(score_threshold = 0.7)
rdocs = retriever.get_relevant_documents("do you have javascript course")
rdocs

[Document(metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}, page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.'),
 Document(metadata={'source': 'Is there any prerequisite for taking this bootcamp ?', 'row': 2}, page_content='prompt: Is there any prerequisite for taking this bootcamp ?\nresponse: Our bootcamp is specifically designed for beginners with no prior experience in this field. The only prerequisite is that you need to have a functional laptop with at least 4GB ram, an internet connection, and a thrill to learn data analysis.'),
 Document(metadata={'source': 'I have never done programming in my life. Can I take this b

As you can see above, the retriever that was created using FAISS and hugging face embedding is now capable of pulling relavant documents from our original CSV file knowledge store. This is very powerful and it will help us further in our project

# Create RetrievalQA chain along with prompt template 🚀

In [77]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on the following pieces of context only.
In the answer try to provide as much text as possible from "response" section in the source document context.
If you can't find the answer in the context, please kindly state "I don't know."


CONTEXT: {context}

QUESTION: {question}"""

PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)


In [78]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs = {"prompt": PROMPT}
)

# We are all set 👍🏼 Let's ask some questions now

In [81]:
chain.invoke("Do you have a plan of lauching blockchain course in the future?")

{'query': 'Do you have a plan of lauching blockchain course in the future?',
 'result': "I don't know. \n",
 'source_documents': [Document(metadata={'source': 'Will the course be upgraded when there are new features in Power BI?', 'row': 27}, page_content='prompt: Will the course be upgraded when there are new features in Power BI?\nresponse: Yes, the course will be upgraded periodically based on the new features in Power BI, and learners who have already bought this course will have free access to the upgrades.'),
  Document(metadata={'source': 'Is there any prerequisite for taking this course?', 'row': 35}, page_content='prompt: Is there any prerequisite for taking this course?\nresponse: The only prerequisite is that you need to have a functional laptop with at least 4GB ram, internet connection and a thrill to learn data analysis.'),
  Document(metadata={'source': 'Is there any prerequisite for taking this bootcamp ?', 'row': 2}, page_content='prompt: Is there any prerequisite for 